In [80]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [92]:
date_range_high = datetime.today().date()
date_range_low = datetime.today().date() - timedelta(days=7)

In [103]:
### webscrape Reuters M&A News

date_filter = date_range_high
page_number = 1
reuters_news_dict = {}

while date_filter >= date_range_low:

    reuters_raw = requests.get("https://www.reuters.com/news/archive/mergersnews?view=page&page={}&pageSize=10".format(page_number))
    reuters_content = reuters_raw.content
    reuters_bs4 = BeautifulSoup(reuters_content)

    reuters_news_block = reuters_bs4.find_all('div', class_ = 'column1 col col-10')
    reuters_news_list  = reuters_news_block[0].find_all('h3', class_ = 'story-title')
    reuters_timestamp_list = reuters_news_block[0].find_all('span', class_ = 'timestamp')

    for i in range(len(reuters_news_list)):
        title = reuters_news_list[i].get_text()
        title = title.split('\n\t\t\t\t\t\t\t\t')[1]

        a_date = reuters_timestamp_list[i].get_text()
        if 'am' in a_date or 'pm' in a_date:
            a_date = datetime.today().date()
        else:
            a_date = datetime.strptime(a_date, "%b %d %Y").date()

        reuters_news_dict[title] = a_date

    date_filter = a_date
    page_number += 1

In [104]:
reuters_news_dict = {title: date for title, date in reuters_news_dict.items() if date >= date_range_low}

In [108]:
### webscrape Seeking Alpha M&A News

page_number = 1
requests.get("https://seekingalpha.com/market-news/m-a?page={}".format(page_number))

<Response [403]>

In [107]:
reuters_news_dict

{'UPDATE 4-Morgan Stanley outperforms rivals with profit beat': datetime.date(2022, 1, 19),
 "UPDATE 4-Chinese regulator denies issuing guidelines on internet giants' investments, fundraisings": datetime.date(2022, 1, 19),
 'CORRECTED-UPDATE 2-Investing app Acorns, Pioneer SPAC scrap $2.2 bln merger (Jan. 18)': datetime.date(2022, 1, 19),
 'EXCLUSIVE-Bain Capital nears $2.27 bln deal to buy French IT services firm Inetum -sources': datetime.date(2022, 1, 19),
 'Morgan Stanley posts higher profit on M&A boom': datetime.date(2022, 1, 19),
 "Epilepsy drug maker Zogenix soars on UCB's potential $2 bln deal": datetime.date(2022, 1, 19),
 'EXCLUSIVE-Bain Capital nears $2.27 bln deal to buy French IT services firm Inetum - sources': datetime.date(2022, 1, 19),
 'UPDATE 1-GSK chief scientist Barron to leave for U.S. biotech start-up': datetime.date(2022, 1, 19),
 'CORRECTED-GSK chief scientist Barron to leave for U.S. biotech start-up': datetime.date(2022, 1, 19),
 'ANALYSIS-Sony faces deep-po